In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline  
import datetime as dt

/Users/fqian/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import cross_val_predict
from sklearn.cross_validation import cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn import svm

#### 1. First clean: only take the useful columns

In [31]:
round3 = pd.read_json("round3_merged.json")
# round3.drop(["Unnamed: 0", "Unnamed: 0.1"],axis=1, inplace=True)
print round3.columns
# round3.head()

round3_clean = round3[["id", "host_id", "host_response_rate", "host_acceptance_rate", 
                       "host_total_listings_count", "monthly_price", "instant_bookable", 
                       "room_type", "bucket_name", "discount_asked", "nightly_price", 
                       "decision", "price_agreed", "discount_agreed", "percent_agreed",
                       "calendars", "price_requested"]]
round3_clean.head()

Index([u'Unnamed: 0', u'account', u'actions', u'availability', u'beds',
       u'bucket_name', u'calendars', u'decision', u'discount_agreed',
       u'discount_asked', u'discount_obtained', u'extra_people',
       u'guests_included', u'host_acceptance_rate', u'host_id',
       u'host_is_superhost', u'host_name', u'host_response_rate',
       u'host_response_time', u'host_total_listings_count', u'host_url', u'id',
       u'instant_bookable', u'listing_url', u'minimum_nights',
       u'monthly_price', u'nightly_plus_guest_price', u'nightly_price',
       u'nightly_price_scraped', u'number_of_reviews', u'openings',
       u'percent_agreed', u'price_agreed', u'price_requested', u'response',
       u'room_type'],
      dtype='object')


,id,host_id,host_response_rate,host_acceptance_rate,host_total_listings_count,monthly_price,instant_bookable,room_type,bucket_name,discount_asked,nightly_price,decision,price_agreed,discount_agreed,percent_agreed,calendars,price_requested
0,755528,3684360,89%,100%,5,None,f,Private room,days3_weeks2,0.9,49,0,NaN,NaN,NaN,"{u'date': {u'56': u'2016-06-06', u'77': u'2016...",40
1,5232121,3339701,100%,100%,5,"$1,500.00",t,Private room,days3_weeks2,0.7,55,0,NaN,NaN,NaN,"{u'date': {u'56': u'2016-06-06', u'77': u'2016...",35
10,4836517,22952210,None,None,1,None,f,Entire home/apt,daysM_weeks2,0.9,50,0,NaN,NaN,NaN,"{u'date': {u'56': u'2016-06-06', u'77': u'2016...",NaN
100,6835052,7768807,100%,100%,2,None,t,Entire home/apt,days1_weeksM,0.6,100,0,NaN,NaN,NaN,"{u'date': {u'56': u'2016-06-06', u'77': u'2016...",NaN
101,4142752,21490002,100%,97%,2,"$4,750.00",f,Entire home/apt,days1_weeks1,0.6,50,0,NaN,NaN,NaN,"{u'date': {u'56': u'2016-06-06', u'77': u'2016...",NaN


#### 2. Add useful variables

In [50]:
calendar_price =  np.array([(np.median(calendar["price_USD"].values()), np.std(calendar["price_USD"].values())) for calendar in round3_clean["calendars"].values])
calendar_median_price = calendar_price[:,0]
calendar_price_std = calendar_price[:,1]
round3_clean["price_median"]=calendar_median_price
round3_clean["price_std"]=calendar_price_std/calendar_median_price
round3_clean.head()

/Users/fqian/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/fqian/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,id,host_id,host_response_rate,host_acceptance_rate,host_total_listings_count,monthly_price,instant_bookable,room_type,bucket_name,discount_asked,nightly_price,decision,price_agreed,discount_agreed,percent_agreed,calendars,price_requested,median_price,price_median,price_std
0,755528,3684360,89%,100%,5,None,f,Private room,days3_weeks2,0.9,49,0,NaN,NaN,NaN,"{u'date': {u'56': u'2016-06-06', u'77': u'2016...",40,49,49,0.132639
1,5232121,3339701,100%,100%,5,"$1,500.00",t,Private room,days3_weeks2,0.7,55,0,NaN,NaN,NaN,"{u'date': {u'56': u'2016-06-06', u'77': u'2016...",35,55,55,0.019470
10,4836517,22952210,None,None,1,None,f,Entire home/apt,daysM_weeks2,0.9,50,0,NaN,NaN,NaN,"{u'date': {u'56': u'2016-06-06', u'77': u'2016...",NaN,52,52,0.040914
100,6835052,7768807,100%,100%,2,None,t,Entire home/apt,days1_weeksM,0.6,100,0,NaN,NaN,NaN,"{u'date': {u'56': u'2016-06-06', u'77': u'2016...",NaN,190,190,0.128456
101,4142752,21490002,100%,97%,2,"$4,750.00",f,Entire home/apt,days1_weeks1,0.6,50,0,NaN,NaN,NaN,"{u'date': {u'56': u'2016-06-06', u'77': u'2016...",NaN,175,175,0.196970


In [103]:
responses = pd.read_csv('Responses Full 0328.csv')
responses.head()

,Unnamed: 0,actions,bucket_name,discount_asked,id,nightly_price,nightly_price_scraped,response,availability,price,discount
0,0,"[<span class=""horizontal-rule-wrapper"" data-re...",days1_weeks1,0.6,8823897,55,$60,"Hi Reuben, my name is Michele and I am interes...",-1,NaN,NaN
1,1,[],days1_weeks1,0.7,7803063,65,$120,No can do. Cleaning fee alone is $50. *******...,0,NaN,NaN
2,2,"[<span class=""horizontal-rule-wrapper"" data-re...",days1_weeks1,0.8,8860745,92,$112,I arrive late* (11pm) *************** Hi Miche...,-1,NaN,NaN
3,3,"[<span class=""horizontal-rule-wrapper"" data-re...",days1_weeks1,0.6,6038278,32,$35,Hi the room is available but the price is firm...,0,NaN,NaN
4,4,[],days1_weeks1,0.7,6400170,74,$130,"Hi Andres, my name is Michele and I am interes...",-1,NaN,NaN


#### Construct 11 variables: 3 types of orphans, 2 types of advances, and 3x2=6 types of interactions

In [104]:
opening_attr = responses["bucket_name"].values
# orphan = []
# advance = []
N = len(opening_attr)
orp_1 = np.zeros(N)
orp_2 = np.zeros(N)
orp_3 = np.zeros(N)
adv_1 = np.zeros(N)
adv_2 = np.zeros(N)

for (i,x) in enumerate(opening_attr):
    if x == "days1_weeks1":
        orp_1[i] = 1
        orp_2[i] = 0
        orp_3[i] = 0
        adv_1[i] = 1
        adv_2[i] = 0
    elif x == "days1_weeks2":
        orp_1[i] = 1
        orp_2[i] = 0
        orp_3[i] = 0
        adv_1[i] = 0
        adv_2[i] = 1
    elif x == "days1_weeksM":
        orp_1[i] = 1
        orp_2[i] = 0
        orp_3[i] = 0
        adv_1[i] = 0
        adv_2[i] = 0
    elif x == "days2_weeks1":
        orp_1[i] = 0
        orp_2[i] = 1
        orp_3[i] = 0
        adv_1[i] = 1
        adv_2[i] = 0
    elif x == "days2_weeks2":
        orp_1[i] = 0
        orp_2[i] = 1
        orp_3[i] = 0
        adv_1[i] = 0
        adv_2[i] = 1
    elif x == "days2_weeksM":
        orp_1[i] = 0
        orp_2[i] = 1
        orp_3[i] = 0
        adv_1[i] = 0
        adv_2[i] = 0
    elif x == "days3_weeks1":
        orp_1[i] = 0
        orp_2[i] = 0
        orp_3[i] = 1
        adv_1[i] = 1
        adv_2[i] = 0
    elif x == "days3_weeks2":
        orp_1[i] = 0
        orp_2[i] = 0
        orp_3[i] = 1
        adv_1[i] = 0
        adv_2[i] = 1
    elif x == "days3_weeksM":
        orp_1[i] = 0
        orp_2[i] = 0
        orp_3[i] = 1
        adv_1[i] = 0
        adv_2[i] = 0
    elif x == "daysM_weeks1":
        orp_1[i] = 0
        orp_2[i] = 0
        orp_3[i] = 0
        adv_1[i] = 1
        adv_2[i] = 0
    elif x == "daysM_weeks2":
        orp_1[i] = 0
        orp_2[i] = 0
        orp_3[i] = 0
        adv_1[i] = 0
        adv_2[i] = 1
    elif x == "daysM_weeksM":
        orp_1[i] = 0
        orp_2[i] = 0
        orp_3[i] = 0
        adv_1[i] = 0
        adv_2[i] = 0
        
responses["orp_1"]=orp_1
responses["orp_2"]=orp_2
responses["orp_3"]=orp_3
responses["adv_1"]=adv_1
responses["adv_2"]=adv_2

In [105]:
responses["orp_1, adv_1"] = responses["orp_1"]*responses["adv_1"]
responses["orp_1, adv_2"] = responses["orp_1"]*responses["adv_2"]
responses["orp_2, adv_1"] = responses["orp_2"]*responses["adv_1"]
responses["orp_2, adv_2"] = responses["orp_2"]*responses["adv_2"]
responses["orp_3, adv_1"] = responses["orp_3"]*responses["adv_1"]
responses["orp_3, adv_2"] = responses["orp_3"]*responses["adv_2"]

In [106]:
responses.head()

,Unnamed: 0,actions,bucket_name,discount_asked,id,nightly_price,nightly_price_scraped,response,availability,price,...,orp_2,orp_3,adv_1,adv_2,"orp_1, adv_1","orp_1, adv_2","orp_2, adv_1","orp_2, adv_2","orp_3, adv_1","orp_3, adv_2"
0,0,"[<span class=""horizontal-rule-wrapper"" data-re...",days1_weeks1,0.6,8823897,55,$60,"Hi Reuben, my name is Michele and I am interes...",-1,NaN,...,0,0,1,0,1,0,0,0,0,0
1,1,[],days1_weeks1,0.7,7803063,65,$120,No can do. Cleaning fee alone is $50. *******...,0,NaN,...,0,0,1,0,1,0,0,0,0,0
2,2,"[<span class=""horizontal-rule-wrapper"" data-re...",days1_weeks1,0.8,8860745,92,$112,I arrive late* (11pm) *************** Hi Miche...,-1,NaN,...,0,0,1,0,1,0,0,0,0,0
3,3,"[<span class=""horizontal-rule-wrapper"" data-re...",days1_weeks1,0.6,6038278,32,$35,Hi the room is available but the price is firm...,0,NaN,...,0,0,1,0,1,0,0,0,0,0
4,4,[],days1_weeks1,0.7,6400170,74,$130,"Hi Andres, my name is Michele and I am interes...",-1,NaN,...,0,0,1,0,1,0,0,0,0,0


In [107]:
inside_airbnb = pd.read_csv('listings_nyc.csv')

shared = inside_airbnb["room_type"].values
inside_airbnb["shared"] = [x=="Private room" for x in shared]
# shared_binary

instant = inside_airbnb["instant_bookable"].values
inside_airbnb["instant"]  = [x=="t" for x in shared]
# instant_binary

In [108]:
available= pd.merge(responses[responses["availability"]!=-1], inside_airbnb, how='inner',left_on=['id'], right_on=['id'])

In [109]:
sample_count = available.groupby("bucket_name")["id"].count()
print len(available)
print sample_count

104
bucket_name
days1_weeks1    10
days1_weeksM     9
days2_weeks1     9
days2_weeks2    12
days2_weeksM    10
days3_weeks1    11
days3_weeks2     5
days3_weeksM     9
daysM_weeks1    10
daysM_weeks2     9
daysM_weeksM    10
Name: id, dtype: int64


In [110]:
cal_param = ["orp_1", "orp_2", "orp_3", "adv_1", "adv_2", "orp_1, adv_1", "orp_1, adv_2", "orp_2, adv_1", "orp_2, adv_2", "orp_3, adv_1", "orp_3, adv_2"]
all_param = cal_param + ["discount_asked", "nightly_price", "shared", "instant"]

In [111]:
X_train = available[all_param].values
Y_train = available["availability"].values

## Logistic Regression

In [112]:
logistic = LogisticRegression()
logistic.fit(X_train, Y_train)
# precisions=cross_validation.cross_val_score(lr, X, y, cv=kf_total, scoring='precision')

lr_scores = -cross_val_score(logistic, X_train, Y_train, scoring='mean_squared_error', cv=5)

#### Coefficient

In [113]:
zip(all_param, np.ravel(logistic.coef_))

[('orp_1', -0.67978310106545903),
 ('orp_2', 0.44899177569948689),
 ('orp_3', 0.48967858479423343),
 ('adv_1', -0.50649697324094611),
 ('adv_2', -0.45148027073001312),
 ('orp_1, adv_1', 0.2947336720609543),
 ('orp_1, adv_2', 0.0),
 ('orp_2, adv_1', -0.069988895069940132),
 ('orp_2, adv_2', 0.43221886011959287),
 ('orp_3, adv_1', -0.04378944953693914),
 ('orp_3, adv_2', 0.41504606143040967),
 ('discount_asked', -0.24696961430142622),
 ('nightly_price', 0.0044412580210620861),
 ('shared', 0.036177417352815128),
 ('instant', 0.0)]

#### Accuracy

In [114]:
logistic.score(X_train, Y_train)

0.69230769230769229

In [115]:
lr_scores

array([ 0.36363636,  0.72727273,  0.55      ,  0.4       ,  0.3       ])

#### Prediction

In [116]:
logistic.predict_proba(X_train)[:5]

array([[ 0.82316984,  0.17683016],
       [ 0.84021617,  0.15978383],
       [ 0.83849245,  0.16150755],
       [ 0.84180909,  0.15819091],
       [ 0.830444  ,  0.169556  ]])

## Random Forest

In [146]:
rf = RandomForestRegressor(n_estimators=100)
rf.fit(X_train, Y_train)
# precisions=cross_validation.cross_val_score(lr, X, y, cv=kf_total, scoring='precision')

rf_scores = -cross_val_score(rf, X_train, Y_train, scoring='mean_squared_error', cv=5)

In [147]:
rf.score(X_train, Y_train)

0.84335812827753132

In [148]:
rf_scores

array([ 0.16735238,  0.23689048,  0.24374762,  0.26832857,  0.28012   ])

In [149]:
rf.predict(X_train)

array([ 0.07,  0.02,  0.  ,  0.16,  0.  ,  0.1 ,  0.74,  0.17,  0.75,
        0.06,  0.02,  0.69,  0.18,  0.05,  0.19,  0.13,  0.15,  0.17,
        0.08,  0.18,  0.04,  0.04,  0.07,  0.01,  0.68,  0.93,  0.11,
        0.93,  0.69,  0.12,  0.26,  0.68,  0.13,  0.31,  0.75,  0.19,
        0.94,  0.85,  0.76,  0.19,  0.13,  0.76,  0.97,  0.85,  0.21,
        0.05,  0.92,  0.92,  0.09,  0.04,  0.28,  0.03,  0.81,  0.08,
        0.05,  0.77,  0.75,  0.3 ,  0.67,  0.03,  0.08,  0.72,  0.8 ,
        0.83,  0.21,  0.19,  0.79,  0.09,  0.69,  0.21,  0.13,  0.26,
        0.74,  0.83,  0.66,  0.65,  0.09,  0.04,  0.09,  0.15,  0.04,
        0.35,  0.24,  0.03,  0.7 ,  0.16,  0.09,  0.68,  0.02,  0.08,
        0.1 ,  0.13,  0.13,  0.13,  0.24,  0.75,  0.68,  0.2 ,  0.06,
        0.74,  0.21,  0.71,  0.73,  0.09])

## SVM

In [134]:
clf = svm.SVC()
clf.fit(X_train, Y_train)  

clf_scores = -cross_val_score(clf, X_train, Y_train, scoring='mean_squared_error', cv=5)

In [135]:
clf.score(X_train, Y_train)

0.82692307692307687

In [136]:
clf_scores

array([ 0.45454545,  0.36363636,  0.4       ,  0.25      ,  0.35      ])

In [137]:
clf.support_vectors_

array([[   1.,    0.,    0., ...,   65.,    1.,    0.],
       [   1.,    0.,    0., ...,   32.,    1.,    0.],
       [   1.,    0.,    0., ...,   48.,    1.,    0.],
       ..., 
       [   0.,    0.,    0., ...,  283.,    0.,    0.],
       [   0.,    0.,    0., ...,   69.,    1.,    0.],
       [   0.,    0.,    0., ...,   72.,    1.,    0.]])